# Ungraded Lab: Transfer Learning
In this lab, you will see how you can use a pre-trained model to achieve good results even with a small training dataset. This is called transfer learning and you do this by leveraging the trained layers of an existing model and adding your own layers to fit your application. For example, you can:

1. just get the convolution layers of one model
2. attach some dense layers onto it
3. train just the dense network
4. evaluate the results
Doing this will allow you to save time building your application because you will essentially skip weeks of training time of very deep networks. You will just use the features it has learned and tweak it for your dataset. Let's see how these are done in the next sections.

## Setup the pretrained model
You will need to prepare pretrained model and configure the layers that you need. For this exercise, you will use the convolution layers of the InceptionV3 architecture as your base model. To do that, you need to:

1. Set the input shape to fit your application. In this case. set it to 150x150x3 as you've been doing in the last few labs.

2. Pick and freeze the convolution layers to take advantage of the features it has learned already.

3. Add dense layers which you will train.

4. Let's see how to do these in the next cells.

First, in preparing the input to the model, you want to fetch the pretrained weights of the InceptionV3 model and remove the fully connected layer at the end because you will be replacing it later. You will also specify the input shape that your model will accept. Lastly, you want to freeze the weights of these layers because they have been trained already.

In [ ]:
# Download the pre-trained weights. No top means it excludes the fully connected layer it uses for classification.
#!wget --no-check-certificate \
#    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

# Set the weights file you downloaded into a variable
local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the base model.
# Set the input shape and remove the dense layers.
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights you downloaded.
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

You can see the summary of the model below. You can see that it is a very deep network. You can then select up to which point of the network you want to use. As Laurence showed in the exercise, you will use up to `mixed_7` as your base model and add to that. This is because the original last layer might be too specialized in what it has learned so it might not translate well into your application. mixed_7 on the other hand will be more generalized and you can start with that for your application. After the exercise, feel free to modify and use other layers to see what the results you get.

In [2]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [2]:
# Choose `mixed_7` as the last layer of your base model
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
last_output

last layer output shape:  (None, 7, 7, 768)


<KerasTensor: shape=(None, 7, 7, 768) dtype=float32 (created by layer 'mixed7')>

In [13]:
last_layer.input

[<KerasTensor: shape=(None, 7, 7, 192) dtype=float32 (created by layer 'activation_154')>,
 <KerasTensor: shape=(None, 7, 7, 192) dtype=float32 (created by layer 'activation_157')>,
 <KerasTensor: shape=(None, 7, 7, 192) dtype=float32 (created by layer 'activation_162')>,
 <KerasTensor: shape=(None, 7, 7, 192) dtype=float32 (created by layer 'activation_163')>]

In [20]:
outputs=[layer for layer in pre_trained_model.inputs]
outputs

[<KerasTensor: shape=(None, 150, 150, 3) dtype=float32 (created by layer 'input_2')>]

## Add dense layers for your classifier
Next, you will add dense layers to your model. These will be the layers that you will train and is tasked with recognizing cats and dogs. You will add a Dropout layer as well to regularize the output and avoid overfitting.

In [3]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 

# Print the model summary. See your dense network connected at the end.
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [16]:
pre_trained_model.input

<KerasTensor: shape=(None, 150, 150, 3) dtype=float32 (created by layer 'input_2')>

In [4]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

# Prepare the dataset

We will use the cats and dogs filtered dataset. Upload it

In [9]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define our example directories and files
base_dir = './cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats') 

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') 

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') 

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Train the model

In [7]:
# Train the model.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 1)

Epoch 1/20
100/100 [==============================] - 25s 179ms/step - loss: 0.3228 - accuracy: 0.8735 - val_loss: 0.1811 - val_accuracy: 0.9330
Epoch 2/20
100/100 [==============================] - 17s 173ms/step - loss: 0.2116 - accuracy: 0.9200 - val_loss: 0.0887 - val_accuracy: 0.9610
Epoch 3/20
100/100 [==============================] - 18s 179ms/step - loss: 0.1911 - accuracy: 0.9290 - val_loss: 0.1128 - val_accuracy: 0.9590
Epoch 4/20
100/100 [==============================] - 15s 154ms/step - loss: 0.2084 - accuracy: 0.9235 - val_loss: 0.1185 - val_accuracy: 0.9560
Epoch 5/20
100/100 [==============================] - 17s 165ms/step - loss: 0.1897 - accuracy: 0.9365 - val_loss: 0.1031 - val_accuracy: 0.9670
Epoch 6/20
100/100 [==============================] - 18s 177ms/step - loss: 0.1848 - accuracy: 0.9345 - val_loss: 0.0945 - val_accuracy: 0.9660
Epoch 7/20
100/100 [==============================] - 17s 167ms/step - loss: 0.1619 - accuracy: 0.9465 - val_loss: 0.1537 - val_ac

In [18]:
from ipywidgets import FileUpload
upload=FileUpload(multiple=True)
display(upload)


FileUpload(value={}, description='Upload', multiple=True)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
for fn in upload.value.keys():
    path=os.path.join('./test/',fn)
    img=image.load_img(path,target_size=(150,150))
    x=image.img_to_array(img)
    x/=255
    x=np.expand_dims(x,axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    
    if classes[0]>0.5:
        print(fn + " is a dog")
        
    else:
        print(fn + " is a cat")
    


[0.00010382]
cat playing.jpg is a cat


In [34]:
import tensorflow as tf
checkpoint_path = "catdog/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback

history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            callbacks=[cp_callback],
            verbose = 1)# Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/20
100/100 [==============================] - 18s 181ms/step - loss: 0.1167 - accuracy: 0.9550 - val_loss: 0.1500 - val_accuracy: 0.9640

Epoch 00001: saving model to catdog\cp.ckpt
Epoch 2/20
100/100 [==============================] - 18s 176ms/step - loss: 0.1217 - accuracy: 0.9590 - val_loss: 0.1167 - val_accuracy: 0.9670

Epoch 00002: saving model to catdog\cp.ckpt
Epoch 3/20
100/100 [==============================] - 18s 176ms/step - loss: 0.1177 - accuracy: 0.9645 - val_loss: 0.1723 - val_accuracy: 0.9620

Epoch 00003: saving model to catdog\cp.ckpt
Epoch 4/20
100/100 [==============================] - 17s 171ms/step - loss: 0.1405 - accuracy: 0.9555 - val_loss: 0.1349 - val_accuracy: 0.9660

Epoch 00004: saving model to catdog\cp.ckpt
Epoch 5/20
100/100 [==============================] - 18s 176ms/step - loss: 0.1192 - accuracy: 0.9600 - val_loss: 0.1369 - val_accuracy: 0.9670

Epoch 00005: saving model to catdog\cp.ckpt
Epoch 6/20
100/100 [==============================

In [35]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

### *Use presaved weights. Load it up from the specified path*

In [6]:
import os
checkpoint_path = "catdog/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [10]:
# Loads the weights
model.load_weights(checkpoint_path)
# Re-evaluate the model
loss, acc = model.evaluate(validation_generator, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

50/50 - 9s - loss: 0.1207 - accuracy: 0.9740
Restored model, accuracy: 97.40%


In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 20

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=2*batch_size)

# Create a new model instance
model = 

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)